In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import scipy.io
import scipy.sparse
import os

# Parameters

In [2]:
# This parameters cell will be overridden by values specified at execution time.
path_data = "../test"
path_out = "../test"
sample_name = "test"

In [3]:
!ls $path_data

devtest.filtered.h5ad              test.doublet-score.pickle
devtest.raw.h5ad                   test.doublets.pickle
devtest_dense.csv                  test.filtered.h5ad
devtest_sparse_counts_barcodes.csv test.raw.h5ad
devtest_sparse_counts_genes.csv    test_dense.csv
devtest_sparse_molecule_counts.mtx test_sparse_counts_barcodes.csv
devtest_sparse_read_counts.mtx     test_sparse_counts_genes.csv
pbmc_1k.filtered.h5ad              test_sparse_molecule_counts.mtx
pbmc_1k.raw.h5ad                   test_sparse_read_counts.mtx


# Dense Count Matrix

## Load

In [4]:
df_dense = pd.read_csv(
    os.path.join(path_data, sample_name + "_dense.csv"),
    index_col=0
)
df_dense

,CLUSTER,ARHGAP33,UPF1,SPPL2B,C19ORF60,TBXA2R,PAF1,MARK4,CCDC124,CEACAM21,...,AC026803.1,METAZOA_SRP.6,SEPT14P19,CTD-3113P16.11,LLNLF-173C4.2,WASH5P,LINC01002,AC008993.2,BISPR,LLNLR-245B6.1
120703436113835,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120703436351717,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120726897253220,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120769892956524,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120778570709861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240695043287403,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
241030031863077,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
241038756308851,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
241038775465902,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Drop the CLUSTER Column

In [5]:
df_dense.drop(columns=["CLUSTER"], inplace=True)

In [6]:
df_dense

,ARHGAP33,UPF1,SPPL2B,C19ORF60,TBXA2R,PAF1,MARK4,CCDC124,CEACAM21,TRAPPC6A,...,AC026803.1,METAZOA_SRP.6,SEPT14P19,CTD-3113P16.11,LLNLF-173C4.2,WASH5P,LINC01002,AC008993.2,BISPR,LLNLR-245B6.1
120703436113835,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120703436351717,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120726897253220,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120769892956524,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120778570709861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240695043287403,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
241030031863077,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
241038756308851,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
241038775465902,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Convert to AnnData

In [7]:
df_dense.index = df_dense.index.astype(str)

In [8]:
adata_filtered = sc.AnnData(df_dense, dtype="int64")

In [9]:
adata_filtered

AnnData object with n_obs × n_vars = 1087 × 1320

In [10]:
adata_filtered.var

""
ARHGAP33
UPF1
SPPL2B
C19ORF60
TBXA2R
...
WASH5P
LINC01002
AC008993.2
BISPR


In [11]:
adata_filtered.obs

""
120703436113835
120703436351717
120726897253220
120769892956524
120778570709861
...
240695043287403
241030031863077
241038756308851
241038775465902


In [12]:
print("FILTERED", adata_filtered)

FILTERED AnnData object with n_obs × n_vars = 1087 × 1320


In [13]:
adata_filtered.to_df()

,ARHGAP33,UPF1,SPPL2B,C19ORF60,TBXA2R,PAF1,MARK4,CCDC124,CEACAM21,TRAPPC6A,...,AC026803.1,METAZOA_SRP.6,SEPT14P19,CTD-3113P16.11,LLNLF-173C4.2,WASH5P,LINC01002,AC008993.2,BISPR,LLNLR-245B6.1
120703436113835,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120703436351717,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120726897253220,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120769892956524,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120778570709861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240695043287403,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
241030031863077,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
241038756308851,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
241038775465902,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Write to Disk

In [14]:
adata_filtered.write_h5ad(
    os.path.join(path_out, sample_name + ".filtered.h5ad"),
)

## Release Memory

In [15]:
del df_dense
del adata_filtered

# Sparse Count Matrix

## Load

In [16]:
mol_counts = scipy.io.mmread(os.path.join(path_data, sample_name + "_sparse_molecule_counts.mtx")).tocsc()

In [17]:
mol_counts

<5941x1347 sparse matrix of type '<class 'numpy.int64'>'
	with 64426 stored elements in Compressed Sparse Column format>

In [18]:
# currently not being used
read_counts = scipy.io.mmread(os.path.join(path_data, sample_name + "_sparse_read_counts.mtx"))

In [19]:
read_counts

<5941x1347 sparse matrix of type '<class 'numpy.int64'>'
	with 64426 stored elements in COOrdinate format>

In [20]:
features = pd.read_csv(os.path.join(path_data, sample_name + "_sparse_counts_genes.csv"), index_col=0, header=None)
features

,1
0,
0,ARHGAP33
1,UPF1
2,SPPL2B
3,C19ORF60
4,TBXA2R
...,...
1342,WASH5P
1343,LINC01002
1344,AC008993.2


In [21]:
barcodes = pd.read_csv(
    os.path.join(path_data, sample_name + "_sparse_counts_barcodes.csv"),
    index_col=0,
    header=None,
    dtype="str"
)[1].values
barcodes

array(['120703424092390', '120703436056350', '120703436113835', ...,
       '241184490047774', '241184504207219', '241184535430878'],
      dtype=object)

## Remove Duplicate Genes

In [22]:
# isolate duplicate genes; column 1 is gene name
dup_genes = features.groupby(1).filter(lambda x: len(x) > 1).groupby(1)

In [23]:
dup_genes.size()

1
METAZOA_SRP    7
dtype: int64

In [24]:
# if there are any duplicate genes
if len(dup_genes) > 0:

    # Aggregate duplicate genes; Uses Column sparse matrix ops
    del_idx = np.array([])
    new_cols = []
    new_genes = features[1].values

    for gene, group in dup_genes:
        idx = group.index.values
        assert len(idx) > 1
        # Get indices to remove duplicate genes and gene columns
        del_idx = np.append(del_idx, idx)
        # Add gene name to end of list
        new_genes = np.append(new_genes, gene)
        # Get aggregated columns to add to end of counts matrix
        new_col = mol_counts[:, idx[0]]
        for i in idx[1:]:
            # Note: csc sum method converts to dense matrix; iterating with '+' is ~10x faster
            new_col = new_col + mol_counts[:, i]
        new_cols.append(new_col)

    # Remove duplicate gene names
    new_genes = np.delete(new_genes, del_idx.astype(int))

    # Concatenate single gene columns with aggregated duplicates
    keep_idx = list(set(features.index.values) - set(del_idx))
    new_counts = scipy.sparse.hstack(
        [mol_counts[:, keep_idx]] + new_cols
    ).tocsc()

    # Readout
    no_cols_removed = len(del_idx) - len(new_cols)
    print(f"{no_cols_removed} gene duplicate columns were removed.")

    features = new_genes
    mol_counts = new_counts
    
else:
    # no duplicate genes
    # column 1 is gene name
    features = features[1]    

6 gene duplicate columns were removed.


## Convert to AnnData

In [25]:
# .X contains molecule counts
adata_raw = sc.AnnData(mol_counts, dtype="int64")

In [26]:
adata_raw.obs_names = barcodes
adata_raw.var_names = features

In [27]:
adata_raw

AnnData object with n_obs × n_vars = 5941 × 1341

In [28]:
adata_raw.obs

""
120703424092390
120703436056350
120703436113835
120703436319462
120703436351717
...
241184489491892
241184489753509
241184490047774
241184504207219


In [29]:
adata_raw.var

""
ARHGAP33
UPF1
SPPL2B
C19ORF60
TBXA2R
...
LINC01002
AC008993.2
BISPR
LLNLR-245B6.1


In [30]:
# molecule counts
adata_raw.X

<5941x1341 sparse matrix of type '<class 'numpy.int64'>'
	with 64426 stored elements in Compressed Sparse Column format>

In [31]:
print("RAW", adata_raw)

RAW AnnData object with n_obs × n_vars = 5941 × 1341


In [32]:
adata_raw.to_df()

,ARHGAP33,UPF1,SPPL2B,C19ORF60,TBXA2R,PAF1,MARK4,CCDC124,CEACAM21,TRAPPC6A,...,AC026803.1,SEPT14P19,CTD-3113P16.11,LLNLF-173C4.2,WASH5P,LINC01002,AC008993.2,BISPR,LLNLR-245B6.1,METAZOA_SRP
120703424092390,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120703436056350,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120703436113835,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120703436319462,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120703436351717,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241184489491892,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
241184489753509,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
241184490047774,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
241184504207219,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Write to Disk

In [33]:
adata_raw.write_h5ad(
    os.path.join(path_out, sample_name + ".raw.h5ad")
)

## Release Memory

In [34]:
del mol_counts
del read_counts
del features
del barcodes
del adata_raw